In [6]:
import requests, datetime, sys
import pandas as pd
import json, os, time
#from spider import crawl

sys.path.append('../../../scripts/')
from s3_support import *

# orgs pulling

In [7]:
DOMAIN = "https://secure.qgiv.com/"
URL = "admin/qgivadmin/statistics/export_tables.php"

post_data = {'key': 'DSQR59VwyFhw21PKDF4K', 'table': 'org'}

In [8]:
print("querying for orgs from secure")
rsp = requests.post(DOMAIN + URL, data=post_data)
org_data = json.loads(rsp.text)
orgs = pd.DataFrame(org_data[0])
print("\tfetched {} entries, storing".format(len(orgs)))

orgs['url'] = orgs['website']
orgs.drop('website', axis=1, inplace=True)

# filter orgs to those that (1) are active and (2) we have a URL
orgs_to_crawl = orgs[(orgs['url']!='')&(orgs['status']==1)].copy()
orgs_to_crawl['url'] = orgs_to_crawl['url'].apply(str)
print("\t{} active orgs with URLs to crawl".format(len(orgs_to_crawl)))

querying for orgs from secure
	fetched 12360 entries, storing
	0 active orgs with URLs to crawl


In [14]:
orgs['status'] = orgs['status'].astype(int)
orgs.tail()

,id,dateCreated,dateLive,dateClosed,mail_addr_state,mail_addr_country,status,reasonClosed,url
12355,449739,2023-01-13 01:20:20,None,None,,US,9,None,http://togetherstrive.org/
12356,449740,2023-01-13 02:00:00,None,None,,US,4,None,http://www.rainbowcityband.com/
12357,449741,2023-01-13 02:52:52,None,None,,US,9,None,https://www.rflgrouphomes.com/
12358,449742,2023-01-13 03:36:36,None,None,,US,9,None,http://www.abileneysa.org/
12359,449743,2023-01-13 04:35:35,None,None,,US,9,None,https://www.labgc.org/


In [15]:
orgs[orgs['status']==1]

,id,dateCreated,dateLive,dateClosed,mail_addr_state,mail_addr_country,status,reasonClosed,url
0,6,2006-05-18 00:00:00,2006-05-18 00:00:00,None,Florida,US,1,None,www.campfire-sunshine.org
2,13,2006-08-22 00:00:00,2006-08-22 00:00:00,None,FL,US,1,None,www.viste.org
7,31,2007-05-18 00:00:00,2007-05-18 00:00:00,None,IA,US,1,None,https://rmhc-eiwi.org/
9,33,2007-06-12 00:00:00,2007-06-12 00:00:00,None,Colorado,US,1,None,www.yeshekhorlousa.com
12,39,2007-12-07 00:00:00,2007-12-07 00:00:00,None,Florida,US,1,None,http://www.givecf.org
...,...,...,...,...,...,...,...,...,...
12300,449684,2022-12-29 03:52:52,2022-12-30 02:13:13,None,AZ,US,1,None,https://aplustuition.org/
12312,449696,2023-01-05 01:26:26,2023-01-12 04:32:32,None,South Dakota,US,1,None,https://siouxfallshabitat.org/
12316,449700,2023-01-06 08:24:24,2023-01-13 09:17:17,None,LA,US,1,None,http://www.acadianabigs.com/
12324,449708,2023-01-06 02:17:17,2023-01-09 02:47:47,None,Connecticut,US,1,None,http://www.b1c.org/


# original

In [9]:
ORG_DATA_PATH = "org_data.csv"
ORG_WEBSITE_DATA_PATH = "org_website_data.csv"

In [3]:
orgs = pd.read_csv(ORG_DATA_PATH)

In [4]:
orgs_to_crawl = orgs[(orgs['url']!='')&(orgs['status']==1)]

In [5]:
len_orgs_to_crawl = len(orgs_to_crawl)

In [6]:
len_orgs_to_crawl

3576

In [8]:
counter = 0
start_time = time.time()

for _, r in orgs_to_crawl.iterrows():
    counter += 1
    
    if counter % 1000 == 0:
        expired_time = (time.time() - start_time) / 60.
        print("{} of {} orgs; {:.4f} minutes".format(counter, len_orgs_to_crawl, expired_time))
        start_time = time.time()

1000 of 3576 orgs; 0.0010 minutes
2000 of 3576 orgs; 0.0009 minutes
3000 of 3576 orgs; 0.0009 minutes


In [12]:
df_web = pd.read_csv(ORG_WEBSITE_DATA_PATH)

In [13]:
df_web.head()

,org,date


# Spider tests

In [15]:
from bs4 import BeautifulSoup
import requests

In [20]:
def get_domain(url):
    split_url = url.split('/')
    for i in range(len(split_url)):
        if 'http' not in split_url[i] and len(split_url[i]) > 0:
            return split_url[i]

In [28]:
url = "www.campfire-sunshine.org"
base_domain = None

In [30]:
protocol = 'http'
if 'https' in url:
    protocol = 'https'
pre_www = 'www' in url

if protocol not in url:
        url = protocol + '://' + url
        
if base_domain is None:
        base_domain = get_domain(url)

In [32]:
resp = requests.get(url, timeout=15)

In [37]:
resp.text

'<!doctype html>\n<html class="no-js" lang="en-US" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:addthis="https://www.addthis.com/help/api-spec" >\n\n<head>\n\t<meta charset="UTF-8" />\n\t<meta http-equiv="x-ua-compatible" content="ie=edge">\n\t<title>Home - Camp Fire Sunshine Central Florida</title>\n\t\n\t<meta name="viewport" content="width=device-width, initial-scale=1">\n\t\n\t<meta property="business:contact_data:email" content="maximizedigitalmedia@gmail.com" />\n\t\n\t<meta property="og:image" content="https://campfire-sunshine.org/wp-content/themes/CampFireCouncilTheme/images/CF_RGB-Logo.jpg" />\n\t<meta property="og:image:secure_url" content="https://campfire-sunshine.org/wp-content/themes/CampFireCouncilTheme/images/CF_RGB-Logo.jpg" />\n\t\n\t<link rel=\'canonical\' href=\'https://campfire-sunshine.org\' />\n\t\n\t<!-- Favicons -->\n\t<link rel="shortcut icon" href="https://campfire-sunshine.org/wp-content/themes/CampFireCouncilTheme/images/favicon.ico" type="image/x-i

In [40]:
soup = BeautifulSoup(resp.text, from_encoding=resp.encoding)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [39]:
resp.encoding

'UTF-8'

In [42]:
soup.find_all('a', href=True)

[<a href="https://www.facebook.com/sunshinecouncil/" target="_blank"><i class="fab fa-facebook-f"></i></a>,
 <a href="https://twitter.com/sunshinecouncil" target="_blank"><i class="fab fa-twitter"></i></a>,
 <a href="https://www.instagram.com/campfire_sunshine/" target="_blank"><i class="fab fa-instagram"></i></a>,
 <a class="button btn-open mobile-menu-button" href="#" id="mobile-menu-button-top">Menu <i class="fas fa-bars"></i></a>,
 <a href="https://campfire-sunshine.org/camp-fire-difference/">About</a>,
 <a href="https://campfire-sunshine.org/board-of-directors/">Board Of Directors</a>,
 <a href="https://campfire-sunshine.org/grants-awards/">Grants/Awards</a>,
 <a href="https://campfire-sunshine.org/annual-reports/">Annual Reports</a>,
 <a href="https://campfire-sunshine.org/sponsors-2/">Friends of Camp Fire</a>,
 <a href="https://campfire-sunshine.org/get-involved/">Get Involved</a>,
 <a href="https://campfire-sunshine.org/our-programs/">Programs</a>,
 <a href="https://campfire-su